# Binder for the Dataverse

This notebook uses a Python environment with pandas to retrieve data from the Dataverse to test some commands for reviewing the data.

## We grab a datafile from the Dataverse and see what we can do with it

In [ ]:
# this does not allow you to set the delimiter, that I am aware of
# import dask.dataframe as dd 
# df = dd.read_csv('https://dataverse.harvard.edu/api/access/datafile/2865473')
# df.head(3)
# df.info()

# basic get
# import csv
# import requests
# r = requests.get('https://dataverse.harvard.edu/api/access/datafile/2865473')
# print(r.text)

DATA_FILE = 'https://demo.dataverse.org/api/access/datafile/2062192?format=original'

# https://demo.dataverse.org/api/access/datafile/2062199?format=original&version=5.2

persistentId=doi:10.70122/FK2/YUUS5M/BWGPCX
# DATA_FILE = 'https://dataverse.harvard.edu/api/access/datafile/7077528'
# with requests.Session() as s:
#     download = s.get(DATA_FILE, stream=True)
#     decoded_content = download.content.decode('utf-8')
#     cr = csv.reader(decoded_content.splitlines()) # , delimiter=',')
#     my_list = list(cr)
#     for row in my_list:
#         print(row)
        
# with requests.get(DATA_FILE, stream=True) as r:
#     lines = (line.decode('utf-8') for line in r.iter_lines())
#     i = 0
#     for row in csv.reader(lines,delimiter='\t'):
#         if i==0 :
#             print(row)
#             print(row[1])
#         i += 1
# For some reason Pandas will not read this file (this does not work without the `storage_options`)
# import pandas as pd
# dfpd = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/2865473', sep='\t')
# dfpd.head(3)

# see https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html on how to group and describe data
import pandas as pd
storage_options = {'User-Agent': 'Mozilla/5.0'}
df = pd.read_stata(DATA_FILE, storage_options=storage_options)
# df.astype('int32').dtypes
print(df)
# print(df[['E03RADRPAKKL','E03RADRPAKKL','E03AGE']].describe())  # prints general stats
# print(df)
# df['E03RADRPAKKL_CAT'] = df['E03RADRPAKKL'].map({
#     "No OA":0,
#     "Questionable":1,
#     "Mild":2,
#     "Moderate":3,
#     "Severe":4,
#     "RA":9
# })
# print(df['E03RADRPAKKL_CAT'])